使用PaddleSeg仓库来构建unet_paddle 人像分割项目。

项目结构
```python
└─-- /unet_paddle
        ├─-- /VOCDevkit/VOC2007   # 人像分割数据集
                ├─-- /ImageSets/Segmentation  # dataset.txt
                ├─-- /JPEGImages     # images
                ├─-- /SegmentationClass # mask
        ├─-- /PaddleSeg    # PaddleSeg仓库
        ├─-- /preweights  # 预训练权重
        ├─-- projs_align.ipynb  # 数据处理

```

PaddleSeg仓库介绍

文件结构
```python
└─-- /PaddleSeg
        ├─-- /configs  # 配置
        ├─-- /deploy   # 部署
        ├─-- /docs     # 文档、教程，建议看
        ├─-- /EISeg    # 交互式分割标注软件
        ├─-- /Matting  # 精细化分割/影像去背/抠图
        ├─-- /paddleseg # paddleseg 包
        ├─-- /test_tipc # 飞桨训推一体认证（TIPC）
        ├─-- /tests    # 模型测试
        ├─-- /tools    # 包含train.py infer.py 等脚本
        ├─-- requirements.txt setup.py  # 构建/配置paddleseg环境
        ├─-- README.md  # PaddleSeg介绍，优先看

```

安装PaddleSeg的依赖
```bash
conda activate pdet
cd path/to/PaddleSeg
pip pip install -r requirements.txt
pip install .
```

数据集格式整理

```python
# 人像分割数据集VOCDevkit
└─-- /dataset/VOCDevkit
        ├─-- /ImageSets/Segmentation # test.txt ...  line:jpg/png文件的文件名
        ├─-- /JPEGImages             # *.jpg rgb
        ├─-- /SegmentationClass      # *.png 单通道png图 0:背景  1:人

# PaddleSeg语义分割数据集格式
└─-- /pseg360  
        ├─-- /images   # *.jpg
        ├─-- /labels   # *.png    #  单通道mask
        ├─-- train.txt # line:images/*.jpg labels/*.png
        ├─-- val.txt
        ├─-- test.txt
```

In [6]:
# 从数据集VOCDevkit中构建pseg360数据集
import os, shutil, tqdm
def getdataset_pseg360(src_root=r"dataset/VOCDevkit/VOC2007", 
                       dst_root=r"PaddleSeg/dataset/pseg360"):
    os.makedirs(dst_root, exist_ok=True)
    src_imgd = os.path.join(src_root, "JPEGImages")
    src_txtd = os.path.join(src_root, "ImageSets/Segmentation")
    src_mskd = os.path.join(src_root, "SegmentationClass")
    
    tgt_imgd = os.path.join(dst_root, "images")
    os.makedirs(tgt_imgd, exist_ok=True)
    tgt_txtd = dst_root
    tgt_mskd = os.path.join(dst_root, "labels")
    os.makedirs(tgt_mskd,exist_ok=True)
    
    def copy_files(set_txt_path, srcd, tgtd, img=True):
        with open(set_txt_path, "r", encoding='utf-8') as f:
            names = f.readlines()
        
        suffix = '.jpg' if img else '.png'
        
        for name in tqdm.tqdm(names):
            name = name.strip()
            shutil.copyfile(os.path.join(srcd, name+suffix), 
                            os.path.join(tgtd, name+suffix))
    
    def gen_txt(src, tgtd):
        with open(src, "r", encoding='utf-8') as fil:
            lines = fil.readlines()
        
        lists = open(os.path.join(tgtd, os.path.basename(src)), "w", encoding='utf-8')
        for ll in lines:
            ll = ll.strip()
            lists.write(f"images/{ll}.jpg labels/{ll}.png\n")

            
      
    for fi in os.listdir(src_txtd):
        if fi != 'trainval.txt':
            src_txtf = os.path.join(src_txtd, fi)
            copy_files(src_txtf, src_imgd, tgt_imgd, img=True)
            copy_files(src_txtf, src_mskd, tgt_mskd, img=False)
            gen_txt(src_txtf, tgt_txtd)
getdataset_pseg360()

- 配置 PaddleSeg/configs/unet
    - 
    - 
    - 
    - 

```yaml
batch_size: 4  # batch_size
iters: 1000    # 模型训练迭代的轮数

train_dataset:  # 训练数据设置
  type: Dataset # 指定加载数据集的类。数据集类的代码在`PaddleSeg/paddleseg/datasets`目录下。
  dataset_root: dataset/pseg360 #数据集路径
  train_path: dataset/pseg360/train_list.txt  #数据集中用于训练的标识文件
  num_classes: 2  # 指定类别个数（背景也算为一类）
  mode: train # 表示用于训练
  transforms: # 模型训练的数据预处理方式。
    - type: ResizeStepScaling #将原始图像和标注图像随机缩放为0.5~2.0倍
      min_scale_factor: 0.5
      max_scale_factor: 2.0
      scale_step_size: 0.25
    - type: RandomPaddingCrop #从原始图像和标注图像中随机裁剪512x512大小
      crop_size: [64, 64]
    - type: RandomHorizontalFlip  #对原始图像和标注图像随机进行水平反转
    - type: RandomDistort #对原始图像进行亮度、对比度、饱和度随机变动，标注图像不变
      brightness_range: 0.5
      contrast_range: 0.5
      saturation_range: 0.5
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

val_dataset:  #验证数据设置
  type: Dataset #指定加载数据集的类。数据集类的代码在`PaddleSeg/paddleseg/datasets`目录下。
  dataset_root: data/optic_disc_seg #数据集路径
  val_path: data/optic_disc_seg/val_list.txt  #数据集中用于验证的标识文件
  num_classes: 2  #指定类别个数（背景也算为一类）
  mode: val #表示用于验证
  transforms: #模型验证的数据预处理的方式
    - type: Normalize #对原始图像进行归一化，标注图像保持不变

optimizer: #设定优化器的类型
  type: SGD #采用SGD（Stochastic Gradient Descent）随机梯度下降方法为优化器
  momentum: 0.9 #设置SGD的动量
  weight_decay: 4.0e-5 #权值衰减，使用的目的是防止过拟合

lr_scheduler: # 学习率的相关设置
  type: PolynomialDecay # 一种学习率类型。共支持12种策略
  learning_rate: 0.01 # 初始学习率
  power: 0.9
  end_lr: 0

loss: #设定损失函数的类型
  types:
    - type: CrossEntropyLoss  #CE损失
  coef: [1, 1, 1] # PP-LiteSeg有一个主loss和两个辅助loss，coef表示权重，所以 total_loss = coef_1 * loss_1 + .... + coef_n * loss_n

model:  #模型说明
  type: PPLiteSeg  #设定模型类别
  backbone:  # 设定模型的backbone，包括名字和预训练权重
    type: STDC2
    pretrained: https://bj.bcebos.com/paddleseg/dygraph/PP_STDCNet2.tar.gz
```

train

```bash
python tools/train.py --config configs/unet/unet_pseg360_main.yml --use_vdl --save_interval 500 --save_dir output

```
```bash
python tools/train.py --config configs/unet/unet_pseg360_main.yml   --do_eval --use_vdl --save_interval 500 --save_dir output

```

infer
```bash
python tools/predict.py --config configs/unet/unet_pseg360_main.yml --model_path output/path/to/model.pdparams --image_path dataset/pseg360/images/xx.jpg --save_dir output/result

```

test
```bash
python tools/val.py --config configs/unet/unet_pseg360_main.yml --model_path output/path/to/model.pdparams 

```

paddleseg  用Dataset 类  -> val 会因为验证集数据 size不一样 而报错